Egune Task for finetuning TTS models using Mongolian Dataset
> Add blockquote



In [1]:
!sudo apt-get install python3.9 python3.9-distutils python3.9-dev -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.9

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9-lib2to3 python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  python3.9 python3.9-dev python3.9-distutils python3.9-lib2to3
  python3.9-minimal
0 upgraded, 9 newly installed, 0 to remove and 38 not upgraded.
Need to get 12.2 MB of archives.
After this operation, 46.6 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-minimal amd64 3.9.24-1+jammy1 [838 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-minimal amd64 3.9.24-1+jammy1 [2,075 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubunt

In [2]:
!python3.9 -m pip install --upgrade pip setuptools wheel

In [10]:
import sys
sys.path.append("/usr/local/lib/python3.9/dist-packages")

In [1]:
#Check runtime on colab jupter notebook
!nvidia-smi

Fri Oct 17 07:41:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#Pull github
!git clone https://github.com/dgduksict/tts-egune.git
%cd tts-egune

Cloning into 'tts-egune'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 51 (delta 23), reused 21 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 47.21 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/tts-egune


In [5]:
#Download needed packages
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 131.8 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 89.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 122.6 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 119.7 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 126.0 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. T

In [11]:
#Import necessary libs
from huggingface_hub import notebook_login, hf_hub_download
from datasets import load_dataset, Audio, get_dataset_config_names
import tarfile, os, pandas as pd

In [2]:
#Hugging face login

notebook_login()

In [3]:
#Define necessary functions
def example():
  return "hello duke"

def preprocess(batch):
    return {
        "input_text": batch["sentence"],
        "speech_array": batch["audio"]["array"],
    }

example()

'hello duke'

In [ ]:
# dataset = load_dataset("mozilla-foundation/common_voice_17_0", "mn", split="train")
# save_dir = "/commonvoice_mn"
# os.makedirs(save_dir, exist_ok=True)

# Extract all tar contents (audio clips + TSV)
# for i, sample in enumerate(dataset):
#     file_path = sample["__key__"]
#     url = sample["__url__"]
#     if url.endswith(".tar") or url.endswith(".tar.gz"):
#         archive_path = os.path.join(save_dir, os.path.basename(url))
#         dataset.download_and_prepare()
#         with tarfile.open(archive_path, "r:*") as tar:
#             tar.extractall(save_dir)
#         break

# print("Extracted dataset to:", save_dir)
# !ls {save_dir}

# print(dataset.features)
# print("Len: ", len(dataset))

In [4]:
dataset = load_dataset("dgduksict/commonvoice-mn", split="train")


print(dataset[0]["path"])  # Example relative path
print(dataset.cache_files)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


common_voice_mn_21799896.mp3
[{'filename': '/root/.cache/huggingface/datasets/dgduksict___commonvoice-mn/default/0.0.0/be751874470047d36224da51e9c788e511464794/commonvoice-mn-train.arrow'}]


In [5]:
# Point to the directory containing your .mp3 files
base_dir = "/root/.cache/huggingface/datasets/dgduksict___commonvoice-mn/default/0.0.0/be751874470047d36224da51e9c788e511464794/"

# Add 'audio' column by decoding the mp3 files
dataset = dataset.map(
    lambda x: {"audio": {"path": os.path.join(base_dir, x["path"])}}
)

# Tell HF that this column is an audio feature (automatically loads the waveform)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [10]:
!pip install -U datasets[audio]

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached multiprocess-0.70.16-py39-none-any.whl.metadata (7.2 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached torch-2.8.0-cp39-cp39-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached aiohttp-3.13.0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached multidict-6.7.0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (5.

In [12]:
dataset[0]["audio"]["array"].shape

ImportError: To support decoding audio data, please install 'torchcodec'.

In [8]:
processed_dataset = dataset.map(preprocess)

Map:   0%|          | 0/2191 [00:00<?, ? examples/s]

ImportError: To support decoding audio data, please install 'torchcodec'.

In [12]:
!ls

finetune.py  LICENCE  notebook.ipynb  README.md  requirements.txt  setup.py


In [13]:
save_dir = "/content/mongolian_tts"
os.makedirs(save_dir, exist_ok=True)

# assuming dataset variable is called `dataset`
metadata_path = os.path.join(save_dir, "metadata.csv")

df = pd.DataFrame({
    "audio_path": [sample["path"] for sample in dataset],
    "sentence": [sample["sentence"] for sample in dataset]
})
df.to_csv(metadata_path, sep="|", header=False, index=False)

print("✅ Metadata saved at:", metadata_path)
print(df.head())

✅ Metadata saved at: /content/mongolian_tts/metadata.csv
                     audio_path  \
0  common_voice_mn_21799896.mp3   
1  common_voice_mn_21799897.mp3   
2  common_voice_mn_21799900.mp3   
3  common_voice_mn_21799907.mp3   
4  common_voice_mn_21799909.mp3   

                                            sentence  
0  хэмээн өөрийгөө тойрч зогссон шавь нартаа тунь...  
1  Тэр хуруун чинээ хүү чинь биднийг мартчихсан ю...  
2  Энэ нь эрдэмтдийн дэвшүүлж байгаа таамаглал бө...  
3       Болж өгвөл сүүдрээсээ хүртэл болгоомжилж яв.  
4  Тиймээс хийсэн үйлдэл бүхэн нь зөв байж тэд бү...  


In [18]:
import shutil
from tqdm import tqdm

audio_dir = os.path.join(save_dir, "clips")
os.makedirs(audio_dir, exist_ok=True)

print(audio_dir)

for sample in tqdm(dataset):
    audio_info = sample["audio"]
    source_path = audio_info["path"]  # ✅ real full path
    dest_path = os.path.join(audio_dir, os.path.basename(source_path))
    if not os.path.exists(dest_path):
        shutil.copy(source_path, dest_path)


/content/mongolian_tts/clips


  0%|          | 0/2191 [00:00<?, ?it/s]


KeyError: 'audio'